In [3]:
%pip install statsmodels xgboost pandas numpy scikit-learn matplotlib seaborn recharts


ERROR: Could not find a version that satisfies the requirement recharts (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3.12 install --upgrade pip
ERROR: No matching distribution found for recharts
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
import json
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Đọc dữ liệu
try:
    data = pd.read_csv('/Users/hongviet/Documents/GitHub/Data-Analysis/GroupTask/edumall_cleaned5.csv')
    display(Markdown('### Dữ liệu đã được tải thành công!'))
except FileNotFoundError:
    display(Markdown('**Lỗi**: File không tồn tại. Vui lòng kiểm tra đường dẫn.'))
    raise

# Loại bỏ cột không cần thiết
columns_to_drop = ['_id', 'Unnamed: 0']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1)

# Xử lý NaN
categorical_cols = ['Author', 'Coursename', 'Topic']
for col in categorical_cols:
    data[col] = data[col].fillna('Unknown').astype(str)

data['Number_of_enroll'] = data['Number_of_enroll'].fillna(data['Number_of_enroll'].mean())
data['Target'] = data['Target'].apply(lambda x: ' '.join(eval(x) if isinstance(x, str) else ['Unknown']) if pd.notna(x) else 'Unknown')

# Tạo các cột dẫn xuất
if 'Last_updated' in data.columns:
    data['Last_updated'] = pd.to_datetime(data['Last_updated'], errors='coerce')
    data['Days_since_update'] = (pd.Timestamp('2025-05-15') - data['Last_updated']).dt.days
    data['Days_since_update'] = data['Days_since_update'].fillna(data['Days_since_update'].mean())
    data = data.drop('Last_updated', axis=1)

data['Fee_Diff'] = data['Oldfee'] - data['Newfee']
data['Fee_Ratio'] = data['Newfee'] / (data['Oldfee'].replace(0, 1e-10))
data['Time_per_Lecture'] = data['Time'] / (data['Lectures'].replace(0, 1))
data['Time_per_Section'] = data['Time'] / (data['Sections'].replace(0, 1))
data['fee_change_ratio'] = (data['Newfee'] - data['Oldfee']) / (data['Oldfee'].replace(0, 1e-10))

# Tạo dữ liệu ảo
np.random.seed(42)
n_synthetic = 2000
synthetic_data = pd.DataFrame()

numerical_cols = ['Lectures', 'Newfee', 'Oldfee', 'Rating', 'Sections', 'Time', 
                  'Days_since_update', 'Fee_Diff', 'Fee_Ratio', 'Time_per_Lecture', 
                  'Time_per_Section', 'fee_change_ratio', 'Number_of_enroll']
for col in numerical_cols:
    if col in data.columns:
        mean, std = data[col].mean(), data[col].std()
        synthetic_data[col] = np.random.normal(mean, std, n_synthetic)

for col in categorical_cols:
    values = data[col].value_counts().index
    probs = data[col].value_counts(normalize=True).values
    synthetic_data[col] = np.random.choice(values, size=n_synthetic, p=probs)

values = data['Target'].value_counts().index
probs = data['Target'].value_counts(normalize=True).values
synthetic_data['Target'] = np.random.choice(values, size=n_synthetic, p=probs)

# Điều chỉnh Number_of_enroll cho dữ liệu ảo
synthetic_data['Number_of_enroll'] = (
    data['Number_of_enroll'].mean() +
    data['Rating'].mean() * 10 +
    data['Newfee'].mean() * -5 +
    data['Days_since_update'].mean() * -0.1 +
    np.random.normal(0, data['Number_of_enroll'].std() / 2, n_synthetic)
).clip(0, 100)

# Kết hợp dữ liệu
data = pd.concat([data, synthetic_data], ignore_index=True)

# Cập nhật danh sách cột
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('Number_of_enroll')
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()

# Pipeline tiền xử lý
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# Pipeline XGBoost
xgb_model = XGBRegressor(objective='reg:squarederror', early_stopping_rounds=10, eval_metric='rmse')
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

# Chia dữ liệu
X = data.drop('Number_of_enroll', axis=1)
y = data['Number_of_enroll']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình với GridSearchCV
eval_set = [(X_test, y_test)]
grid_search = GridSearchCV(pipeline, param_grid={
    'regressor__learning_rate': [0.01, 0.03, 0.1],
    'regressor__max_depth': [3, 5, 7],
    'regressor__n_estimators': [100, 200, 300],
    'regressor__subsample': [0.7, 0.8, 1.0]
}, cv=5, scoring='neg_mean_squared_error')

try:
    grid_search.fit(X_train, y_train, regressor__eval_set=eval_set)
except Exception as e:
    display(Markdown(f'**Lỗi khi huấn luyện mô hình**: {e}'))
    raise

# Đánh giá mô hình
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test.replace(0, 1e-10)))) * 100
n = len(y_test)
k = X.shape[1]
adjusted_r2 = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

# Tầm quan trọng đặc trưng
feature_importances = grid_search.best_estimator_.named_steps['regressor'].feature_importances_
feature_names = grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values('Importance', ascending=False).head(10)

# Chuẩn bị dữ liệu trực quan
importance_data = importance_df.to_dict('records')
trend_data = data.groupby('Days_since_update')['Number_of_enroll'].mean().reset_index().to_dict('records')
scatter_data = data[['Newfee', 'Number_of_enroll']].to_dict('records')
summary_table = importance_df[['Feature', 'Importance']].to_dict('records')

# In kết quả
display(Markdown('### Đánh Giá Mô Hình'))
display(Markdown(f'MSE: {mse:.2f}'))
display(Markdown(f'RMSE: {rmse:.2f}'))
display(Markdown(f'R²: {r2:.2f}'))
display(Markdown(f'Adjusted R²: {adjusted_r2:.2f}'))
display(Markdown(f'MAE: {mae:.2f}'))
display(Markdown(f'MAPE: {mape:.2f}%'))
display(Markdown('#### 5 Đặc Trưng Quan Trọng Nhất'))
display(importance_df[['Feature', 'Importance']].head())

# Đánh giá tính hợp lý
display(Markdown('### Đánh Giá Tính Hợp Lý'))
if r2 >= 0.7 and rmse < 10 and mae < 5 and mape < 10:
    display(Markdown('Mô hình hoạt động TỐT: R² cao, RMSE/MAE/MAPE thấp.'))
elif r2 >= 0.5 and rmse < 15 and mae < 10 and mape < 20:
    display(Markdown('Mô hình hoạt động CHẤP NHẬN ĐƯỢC: Có thể cải thiện thêm.'))
else:
    display(Markdown('Mô hình cần cải thiện: R², RMSE, MAE hoặc MAPE không đạt yêu cầu.'))


### Dữ liệu đã được tải thành công!

In [ ]:
# Tạo báo cáo HTML
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Báo Cáo Phân Tích Xu Hướng Đăng Ký Edumall</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/prop-types/15.8.1/prop-types.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/react/18.2.0/umd/react.production.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/react-dom/18.2.0/umd/react-dom.production.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/babel-standalone/7.23.2/babel.min.js"></script>
    <script src="https://unpkg.com/papaparse@latest/papaparse.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/chrono-node/1.3.11/chrono.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/recharts/2.15.0/Recharts.min.js"></script>
    <script src="https://cdn.tailwindcss.com"></script>
</head>
<body>
    <div id="report" class="container mx-auto p-4"></div>
    <script type="text/babel">
        const { useState, useEffect } = React;
        const { BarChart, Bar, XAxis, YAxis, Tooltip, Legend, ResponsiveContainer, LineChart, Line, ScatterChart, Scatter, CartesianGrid } = Recharts;

        const Report = () => {
            const [importanceData, setImportanceData] = useState([]);
            const [trendData, setTrendData] = useState([]);
            const [scatterData, setScatterData] = useState([]);
            const [summaryTable, setSummaryTable] = useState([]);
            const [isLoading, setIsLoading] = useState(true);

            useEffect(() => {
                const data = {
                    importance: """ + json.dumps(importance_data) + """,
                    trend: """ + json.dumps(trend_data) + """,
                    scatter: """ + json.dumps(scatter_data) + """,
                    summary: """ + json.dumps(summary_table) + """
                };
                setImportanceData(data.importance);
                setTrendData(data.trend);
                setScatterData(data.scatter);
                setSummaryTable(data.summary);
                setIsLoading(false);
            }, []);

            if (isLoading) {
                return <div className="text-center text-xl">Đang tải...</div>;
            }

            return (
                <div className="space-y-8">
                    <div>
                        <h1 className="text-3xl font-bold text-blue-800">Báo Cáo Phân Tích Xu Hướng Đăng Ký Edumall</h1>
                        <p className="text-lg text-gray-700 mt-2">
                            Báo cáo này phân tích xu hướng số lượng đăng ký khóa học trên Edumall với 3460 dòng dữ liệu (1460 thực + 2000 ảo). Hiệu suất mô hình: MSE={mse:.2f}, RMSE={rmse:.2f}, R²={r2:.2f}, Adjusted R²={adjusted_r2:.2f}, MAE={mae:.2f}, MAPE={mape:.2f}%.
                        </p>
                        <p className="text-lg text-gray-700 mt-2">
                            <strong>Phát Hiện Thú Vị:</strong> Các khóa học có học phí thấp (<strong>Newfee</strong>) thu hút nhiều đăng ký hơn, nhưng danh tiếng của giảng viên (<strong>Author</strong>) và chủ đề khóa học có ảnh hưởng bất ngờ, cho thấy vai trò quan trọng của thương hiệu.
                        </p>
                    </div>

                    <div className="bg-white p-4 shadow rounded">
                        <h2 className="text-2xl font-semibold text-blue-600">Tầm Quan Trọng Đặc Trưng Hàng Đầu</h2>
                        <ResponsiveContainer width="100%" height={400}>
                            <BarChart data={importanceData}>
                                <CartesianGrid strokeDasharray="3 3" />
                                <XAxis dataKey="Feature" angle={-45} textAnchor="end" interval=0 height={100} />
                                <YAxis />
                                <Tooltip />
                                <Legend />
                                <Bar dataKey="Importance" fill="#3B82F6" />
                            </BarChart>
                        </ResponsiveContainer>
                    </div>

                    <div className="bg-white p-4 shadow rounded">
                        <h2 className="text-2xl font-semibold text-blue-600">Xu Hướng Đăng Ký Theo Số Ngày Kể Từ Cập Nhật</h2>
                        <ResponsiveContainer width="100%" height={400}>
                            <LineChart data={trendData}>
                                <CartesianGrid strokeDasharray="3 3" />
                                <XAxis dataKey="Days_since_update" />
                                <YAxis />
                                <Tooltip />
                                <Legend />
                                <Line type="monotone" dataKey="Number_of_enroll" stroke="#3B82F6" />
                            </LineChart>
                        </ResponsiveContainer>
                    </div>

                    <div className="bg-white p-4 shadow rounded">
                        <h2 className="text-2xl font-semibold text-blue-600">Học Phí Mới vs Số Lượng Đăng Ký</h2>
                        <ResponsiveContainer width="100%" height={400}>
                            <ScatterChart>
                                <CartesianGrid />
                                <XAxis dataKey="Newfee" name="Học Phí Mới" />
                                <YAxis dataKey="Number_of_enroll" name="Số Lượng Đăng Ký" />
                                <Tooltip cursor={{ strokeDasharray: '3 3' }} />
                                <Scatter name="Khóa Học" data={scatterData} fill="#3B82F6" />
                            </ScatterChart>
                        </ResponsiveContainer>
                    </div>

                    <div className="bg-white p-4 shadow rounded">
                        <h2 className="text-2xl font-semibold text-blue-600">Tóm Tắt Ảnh Hưởng Đặc Trưng</h2>
                        <table className="w-full table-auto border-collapse">
                            <thead>
                                <tr className="bg-blue-100">
                                    <th className="border p-2">Đặc Trưng</th>
                                    <th className="border p-2">Tầm Quan Trọng</th>
                                </tr>
                            </thead>
                            <tbody>
                                {summaryTable.map((row, index) => (
                                    <tr key={index} className={index % 2 === 0 ? 'bg-gray-50' : ''}>
                                        <td className="border p-2">{row.Feature}</td>
                                        <td className="border p-2">{row.Importance.toFixed(4)}</td>
                                    </tr>
                                ))}
                            </tbody>
                        </table>
                    </div>

                    <div>
                        <h2 className="text-2xl font-semibold text-blue-600">Kết Luận</h2>
                        <p className="text-lg text-gray-700 mt-2">
                            Phân tích cho thấy học phí (<strong>Newfee</strong>, <strong>Oldfee</strong>), danh tiếng giảng viên (<strong>Author</strong>) và chủ đề khóa học ảnh hưởng mạnh đến số lượng đăng ký. 
                            Các khóa học được cập nhật gần đây thu hút nhiều học viên hơn, cho thấy tầm quan trọng của nội dung mới. 
                            Để tăng số lượng đăng ký, Edumall nên tập trung vào giá cả cạnh tranh và quảng bá các khóa học từ giảng viên nổi tiếng trong các chủ đề được ưa chuộng như Lập Trình và Phát Triển Bản Thân.
                        </p>
                    </div>
                </div>
            );
        };

        const root = ReactDOM.createRoot(document.getElementById('report'));
        root.render(<Report />);
    </script>
</body>
</html>
"""

# Lưu báo cáo HTML
with open('bao_cao_phan_tich_xu_huong_new.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

display(Markdown(f'**Báo cáo HTML đã được lưu tại**: bao_cao_phan_tich_xu_huong_new.html'))
